Pu pratices his Pytorch skills by writing a classification algorithm using Pytorch. He also uses sklearn's LabelEncoder to transform categorical features. This is based on the tutorial on this page: https://machinelearningmastery.com/use-pytorch-deep-learning-models-with-scikit-learn/

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# read the data
data = pd.read_csv("../data/label-encoder/sonar.csv")
X = data.iloc[:, :60]
y = data.iloc[:, 60]

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)


X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [84]:
# define the model
import torch.nn as nn

class SonarClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(60, 60)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(60, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x

In [85]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

batch_size = 8
learning_rate = 0.01
num_epochs = 30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=True)
train, test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

model = SonarClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

print('debug', len(train_loader))

for epoch in range(num_epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device).view(-1, 1)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()

    if epoch % 6 == 0:
        with torch.no_grad():
            all_preds, all_labels = [], []
            for inputs, targets in test_loader:
                inputs = inputs.to(device)
                targets = targets.to(device).view(-1, 1)
                outputs = model(inputs)
                predicted = torch.round(torch.sigmoid(outputs))
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(targets.cpu().numpy())
                num_correct += (predicted == targets).float().sum().item()
                num_samples += targets.size(0)

            precision = precision_score(all_labels, all_preds)
            recall = recall_score(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds)
            accuracy = accuracy_score(all_labels, all_preds)
            print(f"Epoch: [{epoch}/{num_epochs}], Loss: {loss.item():.2f}, Accuracy: {accuracy * 100:.2f}%, Precision: {precision * 100:.2f}%, Recall: {recall * 100:.2f}%, F1: {f1 * 100:.2f}%")


debug 8
Epoch: [0/30], Loss: 0.70, Accuracy: 52.74%, Precision: 0.00%, Recall: 0.00%, F1: 0.00%


C:\Users\Pu Zhang\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: [6/30], Loss: 0.14, Accuracy: 69.18%, Precision: 62.50%, Recall: 86.96%, F1: 72.73%
Epoch: [12/30], Loss: 0.66, Accuracy: 76.71%, Precision: 74.65%, Recall: 76.81%, F1: 75.71%
Epoch: [18/30], Loss: 0.46, Accuracy: 73.97%, Precision: 80.39%, Recall: 59.42%, F1: 68.33%
Epoch: [24/30], Loss: 0.25, Accuracy: 75.34%, Precision: 78.95%, Recall: 65.22%, F1: 71.43%
